# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from scipy.signal import butter, filtfilt
import random

In [84]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/AveragePooled/split_features_3s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [85]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [86]:
train_data

{'melspectrogram': array([[-0.01764554, -0.97810674, -0.97999614, ..., -0.82831436,
         -0.86151993, -0.9852102 ],
        [-0.0176447 , -0.97808164, -0.9799673 , ..., -0.82414514,
         -0.85394555, -0.982918  ],
        [-0.01764692, -0.97814673, -0.9800371 , ..., -0.81990373,
         -0.8500417 , -0.98001206],
        ...,
        [-0.01758811, -0.57715386, -0.5305197 , ..., -0.86590964,
         -0.8951364 , -0.9996821 ],
        [-0.01772401, -0.58169186, -0.5389059 , ..., -0.8691575 ,
         -0.89791137, -0.99996406],
        [-0.01760773, -0.57361674, -0.5377899 , ..., -0.8711317 ,
         -0.9001475 , -0.99997455]], dtype=float32),
 'mfcc': array([[-1.        , -0.07280675, -0.12869048, ...,  0.02178697,
          0.02212632,  0.04228346],
        [-1.        , -0.08856594, -0.15130392, ...,  0.02659525,
          0.01738   ,  0.04704501],
        [-1.        , -0.10337539, -0.15928885, ...,  0.02455455,
          0.01868342,  0.05395849],
        ...,
        [-0.9

In [87]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [88]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [89]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [90]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [91]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [92]:
display(train_y.shape)
display(train_y[:15])

(13725,)

array([ 7, 12,  4, 17,  3,  9, 14,  3,  0,  0, 10,  2, 12,  3,  1],
      dtype=int32)

In [93]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(13725, 128)

array([-0.0581256 , -0.63822806, -0.6389076 , -0.4953763 , -0.36455613,
       -0.47280535, -0.6928881 , -0.7405781 , -0.7677548 , -0.7374399 ,
       -0.75045246, -0.77124953, -0.7853015 , -0.7958842 , -0.7959697 ,
       -0.81874114, -0.84561497, -0.85084915, -0.8656642 , -0.85950583,
       -0.8056054 , -0.82864517, -0.8715404 , -0.8579188 , -0.87162524,
       -0.8241646 , -0.80134803, -0.8806075 , -0.8931194 , -0.8950114 ,
       -0.89938545, -0.8664572 , -0.90240514, -0.9203411 , -0.9429153 ,
       -0.9389389 , -0.94072586, -0.94264907, -0.95205927, -0.9475349 ,
       -0.9499955 , -0.9482352 , -0.96123224, -0.96147287, -0.9631074 ,
       -0.9561097 , -0.9551953 , -0.94871837, -0.95351607, -0.94429374,
       -0.94507396, -0.93894887, -0.9448011 , -0.94699717, -0.9460722 ,
       -0.94137007, -0.9353987 , -0.9335354 , -0.92789626, -0.92009485,
       -0.9186622 , -0.9323982 , -0.93251014, -0.9347964 , -0.93165   ,
       -0.9263686 , -0.9260132 , -0.9232989 , -0.9198529 , -0.91

'mfcc'

(13725, 20)

...

(13725, 84)

array([-54.3924  , -55.247612, -54.019176, -55.412117, -55.176765,
       -51.641773, -51.608307, -56.090916, -57.880806, -58.957306,
       -57.609154, -57.304836, -56.412144, -55.77131 , -57.49621 ,
       -56.045795, -55.64338 , -56.29201 , -54.53955 , -55.223743,
       -56.586563, -55.49619 , -52.550323, -33.252346, -29.488544,
       -36.211555, -52.73948 , -57.32028 , -59.76588 , -60.917732,
       -62.20523 , -63.252007, -65.78648 , -67.615456, -67.43155 ,
       -65.66191 , -63.76412 , -64.753426, -67.14033 , -66.963356,
       -67.89089 , -69.045784, -68.605064, -69.198074, -70.61911 ,
       -72.82115 , -73.32479 , -74.19051 , -72.36656 , -70.252655,
       -73.62735 , -74.25865 , -72.172646, -70.6866  , -75.13241 ,
       -75.91342 , -75.68041 , -77.47689 , -78.32974 , -78.5558  ,
       -78.68926 , -78.85076 , -78.95114 , -78.8215  , -78.64342 ,
       -78.462425, -78.510155, -78.526566, -78.30219 , -78.121315,
       -77.88373 , -78.19443 , -77.83473 , -77.17658 , -76.501

'id'

(13725,)

'XC214099.mp3'

In [94]:
display(val_y.shape)
display(val_y[:15])

(4231,)

array([12, 18, 12,  1, 17, 16, 19, 16, 17,  8, 11,  4,  1,  9, 10],
      dtype=int32)

In [95]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(4231, 128)

array([-0.01764328, -0.9565721 , -0.93247706, -0.88549644, -0.81249005,
       -0.77785146, -0.76034915, -0.7679908 , -0.7814716 , -0.7643195 ,
       -0.7633203 , -0.76356155, -0.7534367 , -0.7411649 , -0.7278031 ,
       -0.7379763 , -0.74492484, -0.74983066, -0.76243466, -0.76931804,
       -0.78249747, -0.8023202 , -0.8106345 , -0.82639325, -0.8330959 ,
       -0.84451175, -0.849241  , -0.8568743 , -0.86505175, -0.88638127,
       -0.89514667, -0.90498   , -0.9097053 , -0.9165155 , -0.917027  ,
       -0.920565  , -0.9250685 , -0.92276466, -0.9206286 , -0.9269251 ,
       -0.934101  , -0.9314349 , -0.94306   , -0.9392371 , -0.9454409 ,
       -0.9512764 , -0.9590243 , -0.9694562 , -0.97636455, -0.9819564 ,
       -0.98757833, -0.9883887 , -0.99239784, -0.9925326 , -0.9923378 ,
       -0.9928755 , -0.9921703 , -0.98805094, -0.9837086 , -0.98562473,
       -0.9882742 , -0.9882152 , -0.98868024, -0.9876179 , -0.98632133,
       -0.98952055, -0.9845733 , -0.9795667 , -0.98120886, -0.98

'mfcc'

(4231, 20)

...

(4231, 84)

array([-58.28224 , -58.30469 , -60.03862 , -61.265427, -59.94281 ,
       -59.965263, -60.767155, -60.456078, -59.14285 , -58.579327,
       -56.73909 , -57.452988, -59.510807, -58.565063, -58.85057 ,
       -58.601093, -60.21403 , -58.776463, -57.312927, -58.795265,
       -56.85332 , -55.202473, -54.4058  , -53.438473, -52.28848 ,
       -48.160553, -48.251144, -48.59912 , -47.294586, -46.99907 ,
       -46.827312, -46.508904, -48.728092, -48.913437, -48.83963 ,
       -48.400917, -47.28493 , -47.166725, -47.661697, -47.72212 ,
       -47.44012 , -46.3531  , -45.144695, -45.080162, -45.847664,
       -46.067047, -46.776585, -47.623924, -48.124687, -50.001648,
       -51.211334, -52.777264, -53.999992, -55.038715, -55.989574,
       -57.921734, -59.345406, -59.930416, -60.376286, -60.95862 ,
       -61.72118 , -62.10022 , -62.469154, -63.866455, -66.58549 ,
       -67.87702 , -69.35963 , -70.02588 , -69.020294, -68.64077 ,
       -68.66529 , -68.9072  , -69.106926, -68.98829 , -68.596

'id'

(4231,)

'XC383514.mp3'

## **Random Forest Model**

Saving evaluation results

In [96]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [97]:
train_results = {}
val_results = {}

val_scores = {}
k=15

### **With all the features**

In [98]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(13725, 244)

In [99]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(4231, 244)

Fit the model with training data

In [100]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [101]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2888206097849208


In [102]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.26      0.37      0.30       260
           1       0.38      0.54      0.44       424
           2       0.21      0.42      0.28       200
           3       0.37      0.47      0.42       318
           4       0.09      0.16      0.12        73
           5       0.03      0.01      0.01        99
           6       0.00      0.00      0.00        86
           7       0.54      0.26      0.35       224
           8       0.11      0.50      0.18        50
           9       0.16      0.25      0.20        71
          10       0.25      0.18      0.21       146
          11       0.27      0.02      0.04       549
          12       0.34      0.28      0.31       415
          13       0.26      0.14      0.18       160
          14       0.01      0.02      0.01        46
          15       0.27      0.40      0.32       255
          16       0.58      0.65      0.61   

### **With MFCCs only**

In [103]:
training_features = train_features['mfcc']

training_features.shape

(13725, 20)

In [104]:
validation_features = val_features['mfcc']

validation_features.shape

(4231, 20)

In [105]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [106]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3004017962656582


In [107]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.20      0.55      0.30       260
           1       0.33      0.37      0.35       424
           2       0.32      0.43      0.37       200
           3       0.63      0.71      0.67       318
           4       0.02      0.01      0.02        73
           5       0.00      0.00      0.00        99
           6       0.03      0.01      0.02        86
           7       0.38      0.45      0.41       224
           8       0.13      0.52      0.21        50
           9       0.18      0.25      0.21        71
          10       0.29      0.29      0.29       146
          11       0.60      0.08      0.14       549
          12       0.28      0.17      0.21       415
          13       0.26      0.12      0.16       160
          14       0.07      0.26      0.11        46
          15       0.19      0.30      0.23       255
          16       0.63      0.58      0.60   

### **With Mel-Spectrogram Only**

In [108]:
training_features = train_features['melspectrogram']

training_features.shape

(13725, 128)

In [109]:
validation_features = val_features['melspectrogram']

validation_features.shape

(4231, 128)

In [110]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [111]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2666036398014654


In [112]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.22      0.35      0.27       260
           1       0.33      0.47      0.39       424
           2       0.34      0.55      0.42       200
           3       0.50      0.63      0.56       318
           4       0.10      0.16      0.13        73
           5       0.13      0.09      0.11        99
           6       0.00      0.00      0.00        86
           7       0.45      0.22      0.30       224
           8       0.13      0.60      0.21        50
           9       0.25      0.24      0.24        71
          10       0.21      0.18      0.19       146
          11       0.06      0.01      0.01       549
          12       0.20      0.19      0.19       415
          13       0.13      0.07      0.10       160
          14       0.02      0.04      0.03        46
          15       0.24      0.46      0.32       255
          16       0.37      0.39      0.38   

## **With Chroma Only**

In [113]:
training_features = train_features['chroma']

training_features.shape

(13725, 12)

In [114]:
validation_features = val_features['chroma']

validation_features.shape

(4231, 12)

In [115]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [116]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.21696998345544788


In [117]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.30      0.17       260
           1       0.33      0.43      0.38       424
           2       0.09      0.28      0.13       200
           3       0.58      0.59      0.59       318
           4       0.08      0.01      0.02        73
           5       0.71      0.10      0.18        99
           6       0.17      0.01      0.02        86
           7       0.15      0.23      0.18       224
           8       0.01      0.02      0.02        50
           9       0.17      0.11      0.14        71
          10       0.06      0.07      0.06       146
          11       0.16      0.01      0.02       549
          12       0.25      0.07      0.12       415
          13       0.20      0.11      0.14       160
          14       0.03      0.09      0.05        46
          15       0.27      0.41      0.33       255
          16       0.26      0.23      0.25   

## **With CQT Only**

In [118]:
training_features = train_features['cqt']

training_features.shape

(13725, 84)

In [119]:
validation_features = val_features['cqt']

validation_features.shape

(4231, 84)

In [120]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [121]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.18388087922476956


In [122]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.13      0.21      0.16       260
           1       0.44      0.52      0.48       424
           2       0.06      0.12      0.08       200
           3       0.24      0.36      0.28       318
           4       0.02      0.01      0.02        73
           5       0.00      0.00      0.00        99
           6       0.04      0.02      0.03        86
           7       0.28      0.23      0.26       224
           8       0.15      0.48      0.23        50
           9       0.04      0.07      0.05        71
          10       0.13      0.08      0.09       146
          11       0.06      0.00      0.00       549
          12       0.25      0.23      0.24       415
          13       0.14      0.19      0.16       160
          14       0.01      0.04      0.02        46
          15       0.18      0.19      0.19       255
          16       0.22      0.17      0.19   

## **With MFCCs and Mel-Spectrogram**

In [123]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(13725, 148)

In [124]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(4231, 148)

In [125]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [126]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.26565823682344597


In [127]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.24      0.43      0.31       260
           1       0.35      0.48      0.41       424
           2       0.35      0.55      0.42       200
           3       0.52      0.64      0.57       318
           4       0.14      0.21      0.17        73
           5       0.03      0.02      0.02        99
           6       0.00      0.00      0.00        86
           7       0.41      0.21      0.28       224
           8       0.11      0.54      0.19        50
           9       0.17      0.23      0.20        71
          10       0.19      0.17      0.18       146
          11       0.14      0.01      0.02       549
          12       0.25      0.25      0.25       415
          13       0.15      0.09      0.11       160
          14       0.01      0.02      0.01        46
          15       0.25      0.40      0.31       255
          16       0.23      0.26      0.24   

## **With MFCCs and Chroma**

In [128]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(13725, 32)

In [129]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(4231, 32)

In [130]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [131]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31340108721342474


In [132]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.21      0.63      0.32       260
           1       0.37      0.50      0.42       424
           2       0.29      0.33      0.31       200
           3       0.61      0.65      0.63       318
           4       0.07      0.03      0.04        73
           5       0.00      0.00      0.00        99
           6       0.11      0.03      0.05        86
           7       0.37      0.37      0.37       224
           8       0.14      0.50      0.22        50
           9       0.30      0.28      0.29        71
          10       0.31      0.25      0.27       146
          11       0.64      0.08      0.15       549
          12       0.31      0.20      0.24       415
          13       0.31      0.13      0.18       160
          14       0.01      0.04      0.02        46
          15       0.20      0.35      0.26       255
          16       0.61      0.63      0.62   

## **With MFCCs and CQT**

In [133]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(13725, 104)

In [134]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 104)

In [135]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [136]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2358780430158355


In [137]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.17      0.25      0.20       260
           1       0.41      0.54      0.47       424
           2       0.16      0.36      0.22       200
           3       0.30      0.41      0.34       318
           4       0.06      0.08      0.07        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.34      0.28      0.31       224
           8       0.11      0.48      0.18        50
           9       0.14      0.24      0.18        71
          10       0.02      0.01      0.02       146
          11       0.30      0.01      0.03       549
          12       0.31      0.27      0.29       415
          13       0.28      0.19      0.23       160
          14       0.02      0.07      0.03        46
          15       0.19      0.19      0.19       255
          16       0.48      0.48      0.48   

## **With Chroma and CQT**

In [138]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(13725, 96)

In [139]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 96)

In [140]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [141]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2164972819664382


In [142]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.21      0.34      0.26       260
           1       0.40      0.56      0.47       424
           2       0.11      0.24      0.16       200
           3       0.27      0.34      0.30       318
           4       0.03      0.04      0.04        73
           5       0.00      0.00      0.00        99
           6       0.01      0.01      0.01        86
           7       0.31      0.25      0.28       224
           8       0.13      0.46      0.21        50
           9       0.06      0.08      0.07        71
          10       0.06      0.03      0.04       146
          11       0.20      0.01      0.02       549
          12       0.26      0.14      0.19       415
          13       0.25      0.21      0.23       160
          14       0.01      0.02      0.01        46
          15       0.16      0.21      0.18       255
          16       0.43      0.47      0.45   

## **With Mel-Spectrogram and Chroma**

In [143]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(13725, 140)

In [144]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(4231, 140)

In [145]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [146]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.28007563223824156


In [147]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.23      0.42      0.30       260
           1       0.35      0.47      0.40       424
           2       0.34      0.53      0.41       200
           3       0.49      0.58      0.53       318
           4       0.06      0.11      0.08        73
           5       0.15      0.09      0.11        99
           6       0.03      0.01      0.02        86
           7       0.40      0.23      0.29       224
           8       0.12      0.52      0.19        50
           9       0.20      0.28      0.23        71
          10       0.19      0.16      0.18       146
          11       0.09      0.01      0.02       549
          12       0.30      0.20      0.24       415
          13       0.12      0.06      0.08       160
          14       0.00      0.00      0.00        46
          15       0.23      0.48      0.31       255
          16       0.44      0.60      0.51   

## **With Mel-Spectrogram and CQT**

In [148]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(13725, 212)

In [149]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 212)

In [150]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [151]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.25596785629874735


In [152]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.20      0.24      0.22       260
           1       0.35      0.50      0.41       424
           2       0.22      0.47      0.30       200
           3       0.32      0.40      0.35       318
           4       0.08      0.11      0.09        73
           5       0.06      0.02      0.03        99
           6       0.02      0.01      0.01        86
           7       0.64      0.30      0.41       224
           8       0.12      0.52      0.20        50
           9       0.13      0.24      0.17        71
          10       0.27      0.16      0.20       146
          11       0.22      0.02      0.03       549
          12       0.26      0.23      0.25       415
          13       0.17      0.09      0.11       160
          14       0.02      0.07      0.03        46
          15       0.22      0.35      0.27       255
          16       0.63      0.54      0.58   

## **With Mel-Spectrogram, MFCCs, CQT**

In [153]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(13725, 232)

In [154]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(4231, 232)

In [155]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [156]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2703852517135429


In [157]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.22      0.29      0.25       260
           1       0.35      0.51      0.41       424
           2       0.22      0.45      0.29       200
           3       0.35      0.45      0.39       318
           4       0.09      0.14      0.11        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.56      0.28      0.37       224
           8       0.11      0.48      0.18        50
           9       0.14      0.23      0.17        71
          10       0.24      0.16      0.19       146
          11       0.26      0.02      0.04       549
          12       0.29      0.26      0.28       415
          13       0.27      0.15      0.19       160
          14       0.02      0.07      0.04        46
          15       0.24      0.33      0.28       255
          16       0.59      0.63      0.61   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [158]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(13725, 160)

In [159]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(4231, 160)

In [160]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [161]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.28007563223824156


In [162]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.25      0.47      0.33       260
           1       0.37      0.48      0.42       424
           2       0.33      0.54      0.41       200
           3       0.52      0.63      0.57       318
           4       0.11      0.15      0.12        73
           5       0.05      0.04      0.05        99
           6       0.00      0.00      0.00        86
           7       0.39      0.22      0.28       224
           8       0.12      0.50      0.19        50
           9       0.18      0.27      0.22        71
          10       0.18      0.16      0.17       146
          11       0.15      0.02      0.03       549
          12       0.31      0.27      0.29       415
          13       0.14      0.09      0.11       160
          14       0.01      0.02      0.01        46
          15       0.25      0.42      0.31       255
          16       0.28      0.34      0.31   

# Review results from all models

In [163]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.289
3,chroma,1.0,0.217
8,chroma_cqt,1.0,0.216
4,cqt,1.0,0.184
2,melspectrogram,1.0,0.267
9,melspectrogram_chroma,1.0,0.280
12,melspectrogram_chroma_mfcc,1.0,0.280
10,melspectrogram_cqt,1.0,0.256
11,melspectrogram_cqt_mfcc,1.0,0.270
1,mfcc,1.0,0.300


In [164]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.266      0.698
3                    chroma     0.200      0.670
8                chroma_cqt     0.198      0.664
4                       cqt     0.169      0.628
2            melspectrogram     0.238      0.690
9     melspectrogram_chroma     0.251      0.704
10       melspectrogram_cqt     0.240      0.681
11  melspectrogram_cqt_mfcc     0.257      0.707
1                      mfcc     0.285      0.746
6               mfcc_chroma     0.295      0.758
7                  mfcc_cqt     0.218      0.677
5       mfcc_melspectrogram     0.239      0.695
